In [22]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import time
import numpy as np
import tensorflow as tf

from tensorflow.python.framework import graph_util
from tensorflow.python.framework import tensor_shape
from tensorflow.python.platform import gfile
from tensorflow.python.util import compat

In [23]:
def load_graph(model_file):
  graph = tf.Graph()
  graph_def = tf.GraphDef()

  with open(model_file, "rb") as f:
    graph_def.ParseFromString(f.read())
  with graph.as_default():
    tf.import_graph_def(graph_def)

  return graph

In [24]:
def load_labels(label_file):
  label = []
  proto_as_ascii_lines = tf.gfile.GFile(label_file).readlines()
  for l in proto_as_ascii_lines:
    label.append(l.rstrip())
  return label

In [25]:
def read_tensor_from_image_file(file_name, input_height=299, input_width=299,
  input_mean=0, input_std=255):
  input_name = "file_reader"
  output_name = "normalized"
  file_reader = tf.read_file(file_name, input_name)
  if file_name.endswith(".png"):
    image_reader = tf.image.decode_png(file_reader, channels = 3,
                                       name='png_reader')
  elif file_name.endswith(".gif"):
    image_reader = tf.squeeze(tf.image.decode_gif(file_reader,
                                                  name='gif_reader'))
  elif file_name.endswith(".bmp"):
    image_reader = tf.image.decode_bmp(file_reader, name='bmp_reader')
  else:
    image_reader = tf.image.decode_jpeg(file_reader, channels = 3,
                                        name='jpeg_reader')
  float_caster = tf.cast(image_reader, tf.float32)
  dims_expander = tf.expand_dims(float_caster, 0);
  resized = tf.image.resize_bilinear(dims_expander, [input_height, input_width])
  normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
  sess = tf.Session()
  result = sess.run(normalized)

  return result

In [32]:
file_name = "/home/minato/217.jpg"
model_file = "../tf_files/inception_retrained_graph_new_2.pb"
label_file = "../tf_files/inception_retrained_labels_new_2.txt"
input_height = 299
input_width = 299
input_mean = 128
input_std = 128
input_layer = "Mul"
output_layer = "final_result"

input_name = "import/" + input_layer
output_name = "import/" + output_layer
graph = load_graph(model_file)
input_operation = graph.get_operation_by_name(input_name);
output_operation = graph.get_operation_by_name(output_name);



In [33]:
t = read_tensor_from_image_file(file_name,
                              input_height=input_height,
                              input_width=input_width,
                              input_mean=input_mean,
                              input_std=input_std)


with tf.Session(graph=graph) as sess:
  start = time.time()
  results = sess.run(output_operation.outputs[0],
                  {input_operation.outputs[0]: t})
  end=time.time()

results = np.squeeze(results)
print(results)
top_k = results.argsort()[-5:][::-1]
labels = load_labels(label_file)

print('\nEvaluation time (1-image): {:.3f}s\n'.format(end-start))

for i in top_k:
    print(labels[i], results[i])

[2.6534995e-04 9.7111352e-02 6.7729540e-03 1.2051094e-03 7.1189759e-05
 1.0385051e-03 3.6110898e-04 1.0445697e-03 5.1032235e-05 4.3704110e-04
 3.7048522e-02 6.0789072e-04 8.5274613e-01 4.5428157e-04 3.6363071e-04
 4.2129910e-04]

Evaluation time (1-image): 0.516s

point shoot digital cameras 0.8527461
accessory kits 0.09711135
memory 0.037048522
camera batteries 0.006772954
computers accessories 0.0012051094


In [20]:
import os
count = 0
for root, directories, filenames in os.walk('/home/minato/datasets_new/'):
    t1 = None
    arr = []    
    for filename in filenames:
        count = count + 1
        file_name=os.path.join(root,filename)
        if(count==1):
            t1 = read_tensor_from_image_file(file_name,
                                  input_height=input_height,
                                  input_width=input_width,
                                  input_mean=input_mean,
                                  input_std=input_std)
            arr = [t1]
        else:
            t1 = read_tensor_from_image_file(file_name,
                                  input_height=input_height,
                                  input_width=input_width,
                                  input_mean=input_mean,
                                  input_std=input_std)
            arr = tf.concat(arr, [t1])
            
    tf.shape(arr)

TypeError: Expected int32, got array([[[[0.984375, 0.984375, 0.984375],
         [0.984375, 0.984375, 0.984375],
         [0.984375, 0.984375, 0.984375],
         ...,
         [0.984375, 0.984375, 0.984375],
         [0.984375, 0.984375, 0.984375],
         [0.984375, 0.984375, 0.984375]],

        [[0.984375, 0.984375, 0.984375],
         [0.984375, 0.984375, 0.984375],
         [0.984375, 0.984375, 0.984375],
         ...,
         [0.984375, 0.984375, 0.984375],
         [0.984375, 0.984375, 0.984375],
         [0.984375, 0.984375, 0.984375]],

        [[0.984375, 0.984375, 0.984375],
         [0.984375, 0.984375, 0.984375],
         [0.984375, 0.984375, 0.984375],
         ...,
         [0.984375, 0.984375, 0.984375],
         [0.984375, 0.984375, 0.984375],
         [0.984375, 0.984375, 0.984375]],

        ...,

        [[0.984375, 0.984375, 0.984375],
         [0.984375, 0.984375, 0.984375],
         [0.984375, 0.984375, 0.984375],
         ...,
         [0.984375, 0.984375, 0.

In [ ]:

        with tf.Session(graph=graph) as sess:
            start = time.time()
            results = sess.run(output_operation.outputs[0],
                          {input_operation.outputs[0]: t})
            end=time.time()

        results = np.squeeze(results)
        print(results)
        top_k = results.argsort()[-5:][::-1]
        labels = load_labels(label_file)

        print('\nEvaluation time (1-image): {:.3f}s\n'.format(end-start))

        for i in top_k:
            print(labels[i], results[i])